In [15]:
#from tools.data_handler import Sig
from _KS_module import *
import numpy as np
import pickle
import sys
from tqdm import tqdm_notebook

output_path = './Sig_KS_results'

def printProgress (iteration, total, prefix = '', suffix = '', decimals = 1, barLength = 100):
    formatStr = "{0:." + str(decimals) + "f}"
    percent = formatStr.format(100 * (iteration / float(total)))
    filledLength = int(round(barLength * iteration / float(total)))
    bar = '#' * filledLength + '-' * (barLength - filledLength)
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percent, '%', suffix)),
    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()
    
def get_data_dic(partition_type):
#     nonlocal case

    path = '%s-dataset%02d' % (partition_type, 0)
    dataset = pd.read_pickle('./data/datasets/sig/'+path)

    datas = dataset['data']
    datas = scale_timestamp(datas)
    targets = dataset['target']
    notes = dataset['note']

    data_dic = {(target, note): []
                for note in notes for target in targets}

    for i, df in enumerate(datas):
        # df = diff_dataframe(df)
        data_dic[targets[i], notes[i]].append(df)
        printProgress(i, len(datas), 'Load {} data :'.format(partition_type), '', 1, 50)
    print("")
    return data_dic

tr_dic = get_data_dic('tr')
te_dic = get_data_dic('te')


Load tr data : |##################################################| 100.0% 
Load te data : |##################################################| 100.0% 


In [55]:
path = '%s-dataset%02d' % ('tr', 0)
dataset = pd.read_pickle('./data/datasets/sig/'+path)


In [74]:
sig_list = sorted(list(set(dataset['note'])))
sig_list

[('sig-en', 0),
 ('sig-en', 1),
 ('sig-en', 2),
 ('sig-en', 3),
 ('sig-en', 4),
 ('sig-en', 5),
 ('sig-en', 6),
 ('sig-en', 7),
 ('sig-en', 8),
 ('sig-en', 9),
 ('sig-fi', 0),
 ('sig-fi', 1),
 ('sig-fi', 2),
 ('sig-fi', 4),
 ('sig-fi', 5),
 ('sig-fi', 6),
 ('sig-fi', 8),
 ('sig-fi', 9),
 ('sig-ko', 0),
 ('sig-ko', 2),
 ('sig-ko', 3),
 ('sig-ko', 4),
 ('sig-ko', 5),
 ('sig-ko', 7),
 ('sig-ko', 8),
 ('sig-ko', 9)]

In [77]:
tr_dic.keys()

dict_keys([('39', ('sig-en', 0)), ('29', ('sig-en', 0)), ('20', ('sig-en', 0)), ('30', ('sig-en', 0)), ('28', ('sig-en', 0)), ('36', ('sig-en', 0)), ('13', ('sig-en', 0)), ('44', ('sig-en', 0)), ('48', ('sig-en', 0)), ('5', ('sig-en', 0)), ('11', ('sig-en', 0)), ('15', ('sig-en', 0)), ('25', ('sig-en', 0)), ('23', ('sig-en', 0)), ('27', ('sig-en', 0)), ('6', ('sig-en', 0)), ('38', ('sig-en', 0)), ('19', ('sig-en', 0)), ('18', ('sig-en', 0)), ('21', ('sig-en', 0)), ('10', ('sig-en', 0)), ('2', ('sig-en', 0)), ('26', ('sig-en', 0)), ('34', ('sig-en', 0)), ('46', ('sig-en', 0)), ('49', ('sig-en', 0)), ('45', ('sig-en', 0)), ('43', ('sig-en', 0)), ('8', ('sig-en', 0)), ('50', ('sig-en', 0)), ('35', ('sig-en', 0)), ('9', ('sig-en', 0)), ('4', ('sig-en', 0)), ('22', ('sig-en', 0)), ('3', ('sig-en', 0)), ('37', ('sig-en', 0)), ('47', ('sig-en', 0)), ('16', ('sig-en', 0)), ('17', ('sig-en', 0)), ('31', ('sig-en', 0)), ('32', ('sig-en', 0)), ('7', ('sig-en', 0)), ('41', ('sig-en', 0)), ('14', (

In [78]:
tr_dic[('1',('sig-en',0))]

[    timeInterval action.0        x.0        y.0      gx.0      gy.0      gz.0  \
 0       1.379035     DOWN  169.27734  257.15625 -0.036176 -0.206428  0.012270   
 1       1.669677     MOVE  169.01231  258.79060 -0.158960 -0.047603  0.029985   
 2       1.678435     MOVE  168.75000  265.83325 -0.294572 -0.062264  0.084352   
 3       1.655669     MOVE  169.27140  279.82825 -0.202331 -0.070205  0.036704   
 4       1.670281     MOVE  172.92601  371.21313 -0.022126 -0.044549  0.035483   
 5       1.661208     MOVE  181.84943  453.26500  0.110432 -0.010340  0.017768   
 6       1.733319     MOVE  194.98607  538.20460  0.136088  0.020203 -0.012165   
 7       1.640146     MOVE  208.35767  614.60474  0.103101  0.053190 -0.034156   
 8       1.642419     MOVE  226.21010  674.96204  0.066449  0.091674 -0.037210   
 9       1.656523     MOVE  242.83275  715.12110  0.039571  0.061742 -0.049428   
 10      0.666662     MOVE  251.59903  733.15027  0.068282  0.012873 -0.026215   
 11     13.96542

In [ ]:
list(tr_dic.keys())[0]

In [ ]:
df = tr_data.copy()

table = df['data'][1000].copy()

In [ ]:
np.array(te_data['note'])

In [ ]:
te_data['note'][-10:]

In [ ]:
# 실험 종류, 번호, 도형 아이디, 도형 인덱스
log_df = sig.log_lookup['sig-fi']['40'][4][4]
log_df[:5]